# Collect data from Spotify and Last.fm

In [1]:
from crawler import Crawler

crawler = Crawler("127.0.0.1", 5000)

In [ ]:
crawler.collect_random_tracks()

Huge party playlist (500+ hours)

In [ ]:
crawler.collect_tracks_from_playlist("69fEt9DN5r4JQATi52sRtq")

refreshing spotify credentials at _retrieve_playlist_tracks
Analyzing 50 tracks ...
No tags 50i4g5QRKsw0jzI2IMvtmC Universe of Calm ['8D Relaxium']
No tags 3zmY99mQqB0lBifOwBzHHy Charleston Charleston ['Damien Poyard']
No tags 7fvOUyRi8q9y4IrQPTIxQ3 Top Of The World / Take Me Home Country Roads / Hey Good Looking ['The Diamond Accordion Band']
No tags 4IBlIZtU6v7SnCNf7kdCgx Three Little Birds - Acoustic ['Richard Watts']
No tags 1FvOd1pzW9c1Cdq0CWtO7x Hanggang Kailan (Umuwi Ka Na Baby) ['Angelina Cruz']
No tags 1mIjg3nbumqPee6BjWpq0F Corre a Sus Pies ['July Ann Alvarado', 'Héctor Delgado']
No tags 3b8GsO4H7M5jC3skXctKby A Él Sea la Gloria - En Vivo ['Karen Rivera', 'Ministerio Sion']
No tags 2tarv1UbVt0c4ygyi1D8NW ¡Oh, Tu Fidelidad! ['Addi Yin', 'Hugo Yin']
No tags 3fi9zB1ojXw9Rv1NbR043q Guatemala Shawty ['lonelyrari']
No tags 7Gha6uktqprgG82ZSMojQ5 To Ba Ma Bash ['Rail 47']
No tags 1nLaAGaf7LToTcuy7vhJb5 Il est bon de louer Dieu ['Dorcas Kaja']
refreshing spotify credentials at _enric